In [77]:
import json
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [79]:
# Загрузка данных из JSON-файла
json_file_path = "cities_data.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    cities_data = json.load(file)

Извлечение данных из json файла было разделено на 2 части (первые 200 кортежей, остальные записи), так как клиент Open-Meteo API не позволяет делать все запросы одним разом (видимо для него это многовато)

In [82]:
# Извлечение данных из первой половины JSON
names = [city["name"] for city in cities_data[:200]]
countries = [city["country"] for city in cities_data[:200]]
latitudes = [city["latitude"] for city in cities_data[:200]]
longitudes = [city["longitude"] for city in cities_data[:200]]

In [84]:
# Инициализация списка для хранения данных
data_records = []


In [86]:
# Настройка клиента Open-Meteo API с кэшем и повторными попытками
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)
# URL API
url = "https://archive-api.open-meteo.com/v1/archive"

In [88]:

params = {
    "latitude": latitudes,
    "longitude": longitudes,
    "start_date": "2020-01-01",
    "end_date": "2021-12-31",
    "hourly": ["temperature_2m", "apparent_temperature", "precipitation", "wind_speed_10m", "wind_direction_10m"]
}

responses = openmeteo.weather_api(url, params=params)


# Обработка данных для каждого местоположения
for i, response in enumerate(responses):
    name = names[i]
    country = countries[i]
    latitude = latitudes[i]
    longitude = longitudes[i]

    print(f"Обработка данных для {name}, {country}: {latitude}°N, {longitude}°E")

    # Извлечение почасовых данных
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
    
    timestamps = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Формирование записей для DataFrame
    for time, temp, apparent_temp, precip, wind_speed, wind_dir in zip(
        timestamps, hourly_temperature_2m, hourly_apparent_temperature, hourly_precipitation,
        hourly_wind_speed_10m, hourly_wind_direction_10m
    ):
        data_records.append({
            "date": time,
            "country": country,
            "city": name,
            "latitude": latitude,
            "longitude": longitude,
            "temperature_2m": temp,
            "apparent_temperature": apparent_temp,
            "precipitation": precip,
            "wind_speed_10m": wind_speed,
            "wind_direction_10m": wind_dir
        })


Обработка данных для Берлин, Германия: 52.52°N, 13.41°E
Обработка данных для Ален, Германия: 51.76°N, 7.89°E
Обработка данных для Ален, Германия: 48.84°N, 10.09°E
Обработка данных для Арнсберге, Германия: 51.38°N, 8.08°E
Обработка данных для Аугсбург, Германия: 48.37°N, 10.9°E
Обработка данных для Ахен, Германия: 50.78°N, 6.08°E
Обработка данных для Ашаффенбург, Германия: 49.98°N, 9.15°E
Обработка данных для Бад-Зальцуфлен, Германия: 52.09°N, 8.74°E
Обработка данных для Бад-Хомбург, Германия: 50.23°N, 8.62°E
Обработка данных для Баден-Баден, Германия: 48.76°N, 8.24°E
Обработка данных для Байройт, Германия: 49.95°N, 11.58°E
Обработка данных для Бамберг, Германия: 49.9°N, 10.9°E
Обработка данных для Бергем, Германия: 50.96°N, 6.64°E
Обработка данных для Бергиш Гладбах, Германия: 50.99°N, 7.13°E
Обработка данных для Бергкамен, Германия: 51.62°N, 7.64°E
Обработка данных для Билефельд, Германия: 52.03°N, 8.53°E
Обработка данных для Бонн, Германия: 50.73°N, 7.1°E
Обработка данных для Ботроп,

In [218]:
# Извлечение данных из второй половины JSON
names = [city["name"] for city in cities_data[200:]]
countries = [city["country"] for city in cities_data[200:]]
latitudes = [city["latitude"] for city in cities_data[200:]]
longitudes = [city["longitude"] for city in cities_data[200:]]

params = {
    "latitude": latitudes,
    "longitude": longitudes,
    "start_date": "2020-01-01",
    "end_date": "2021-12-31",
    "hourly": ["temperature_2m", "apparent_temperature", "precipitation", "wind_speed_10m", "wind_direction_10m"]
}

responses = openmeteo.weather_api(url, params=params)


# Обработка данных для каждого местоположения
for i, response in enumerate(responses):
    name = names[i]
    country = countries[i]
    latitude = latitudes[i]
    longitude = longitudes[i]

    print(f"Обработка данных для {name}, {country}: {latitude}°N, {longitude}°E")

    # Извлечение почасовых данных
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
    
    timestamps = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Формирование записей для DataFrame
    for time, temp, apparent_temp, precip, wind_speed, wind_dir in zip(
        timestamps, hourly_temperature_2m, hourly_apparent_temperature, hourly_precipitation,
        hourly_wind_speed_10m, hourly_wind_direction_10m
    ):
        data_records.append({
            "date": time,
            "country": country,
            "city": name,
            "latitude": latitude,
            "longitude": longitude,
            "temperature_2m": temp,
            "apparent_temperature": apparent_temp,
            "precipitation": precip,
            "wind_speed_10m": wind_speed,
            "wind_direction_10m": wind_dir
        })


Обработка данных для Кавала, Греция: 40.94°N, 24.41°E
Обработка данных для Каламария, Греция: 40.58°N, 22.95°E
Обработка данных для Каламата, Греция: 37.04°N, 22.11°E
Обработка данных для Каллифея, Греция: 37.95°N, 23.7°E
Обработка данных для Ламия, Греция: 38.9°N, 22.43°E
Обработка данных для Лариса, Греция: 39.64°N, 22.42°E
Обработка данных для Перистерион, Греция: 38.02°N, 23.69°E
Обработка данных для Петроуполис, Греция: 38.04°N, 23.68°E
Обработка данных для Пирей, Греция: 37.94°N, 23.65°E
Обработка данных для Родос, Греция: 36.44°N, 28.22°E
Обработка данных для Салоники, Греция: 40.64°N, 22.93°E
Обработка данных для Серре, Греция: 41.09°N, 23.55°E
Обработка данных для Трикала, Греция: 39.55°N, 21.77°E
Обработка данных для Халкида, Греция: 38.46°N, 23.6°E
Обработка данных для Ханья, Греция: 35.51°N, 24.03°E
Обработка данных для Берн, Швейцария: 46.95°N, 7.45°E
Обработка данных для Базель, Швейцария: 47.56°N, 7.57°E
Обработка данных для Винтертур, Швейцария: 47.51°N, 8.72°E
Обработк

In [220]:
# Создание итогового DataFrame
final_dataframe = pd.DataFrame(data_records)

In [222]:
final_dataframe.head()

,date,country,city,latitude,longitude,temperature_2m,apparent_temperature,precipitation,wind_speed_10m,wind_direction_10m
0,2020-01-01 00:00:00+00:00,Германия,Берлин,52.52,13.41,2.1085,-1.317814,0.0,10.308831,282.094727
1,2020-01-01 01:00:00+00:00,Германия,Берлин,52.52,13.41,1.8085,-1.758666,0.0,11.159999,270.000000
2,2020-01-01 02:00:00+00:00,Германия,Берлин,52.52,13.41,1.2085,-2.283353,0.0,10.137691,276.115417
3,2020-01-01 03:00:00+00:00,Германия,Берлин,52.52,13.41,0.3585,-3.139605,0.0,9.422101,276.581848
4,2020-01-01 04:00:00+00:00,Германия,Берлин,52.52,13.41,-0.0415,-3.608093,0.0,9.422101,276.581848


In [224]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022992 entries, 0 to 6022991
Data columns (total 10 columns):
 #   Column                Dtype              
---  ------                -----              
 0   date                  datetime64[ns, UTC]
 1   country               object             
 2   city                  object             
 3   latitude              float64            
 4   longitude             float64            
 5   temperature_2m        float32            
 6   apparent_temperature  float32            
 7   precipitation         float32            
 8   wind_speed_10m        float32            
 9   wind_direction_10m    float32            
dtypes: datetime64[ns, UTC](1), float32(5), float64(2), object(2)
memory usage: 344.6+ MB


In [226]:
final_dataframe.to_csv('weather_in_cities_2020.csv', index=False)

print("Данные были сохранены в файл 'weather_in_cities.csv', папка data/source")

Данные были сохранены в файл 'weather_in_cities.csv', папка data/source
